<a href="https://colab.research.google.com/github/hinekobot/Tensorflow2.0_sample/blob/master/01_LinearRegressionSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow2.0  
- Google Colab上での操作を想定
- cloneして中身を変える用の空ファイル

In [1]:
!pip install tensorflow==2.0.0-alpha0

    100% |████████████████████████████████| 79.9MB 423kB/s 
    100% |████████████████████████████████| 61kB 21.9MB/s 
    100% |████████████████████████████████| 3.0MB 9.9MB/s 
    100% |████████████████████████████████| 419kB 20.6MB/s 
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


In [0]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y +2


## Linear Regression
### by Using Normal Equation

In [0]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
#theta = tf.matmul(tf.matmul(tf.matrix_inverse(ft.matmul(XT, X)), XT), y)
theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)

In [18]:
theta

<tf.Tensor: id=42, shape=(9, 1), dtype=float32, numpy=
array([[-3.7144665e+01],
       [ 4.3637392e-01],
       [ 9.3933418e-03],
       [-1.0714764e-01],
       [ 6.4542186e-01],
       [-4.1016874e-06],
       [-3.7816020e-03],
       [-4.2341197e-01],
       [-4.3682083e-01]], dtype=float32)>

### Linear Regression by Using Stochastic Descent
- 解説書だと`tf.assign()`を使っているが、Tensorflow2.0では削除されている
- 仕方ないのでループで計算させている

In [0]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
#scaler = StandardScaler()
scaler = MinMaxScaler()

scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [51]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
#theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)

for epoch in range(n_epochs):
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
  theta = theta - learning_rate * gradients
  y_pred = tf.matmul(X, theta,name='theta')
  error = y_pred - y
  mse = tf.reduce_mean(tf.square(error),name='mse')
  gradients = 2/m * tf.matmul(tf.transpose(X), error)

best_theta = theta.numpy()

Epoch 0 MSE= 6.105645
Epoch 100 MSE= 1.481247
Epoch 200 MSE= 1.1728331
Epoch 300 MSE= 1.1158499
Epoch 400 MSE= 1.0773189
Epoch 500 MSE= 1.0433859
Epoch 600 MSE= 1.0128089
Epoch 700 MSE= 0.9851367
Epoch 800 MSE= 0.96002233
Epoch 900 MSE= 0.93716973


### Linear Regression by Using Stochastic Descent: Auto differentiation

In [54]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
#theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
#gradients = tf.gradients(mse, [theta])[0]
gradients = tf.GradientTape(mse, [theta])[0]

for epoch in range(n_epochs):
  if epoch % 100 == 0:
    print('Epoch', epoch, 'MSE=', mse.numpy())
    
  theta = theta - learning_rate * gradients
  y_pred = tf.matmul(X, theta,name='theta')
  error = y_pred - y
  mse = tf.reduce_mean(tf.square(error),name='mse')
  #   gradients = 2/m * tf.matmul(tf.transpose(X), error)
  gradients = tf.GradientTape(mse, [theta])[0]

best_theta = theta.numpy()

TypeError: ignored